In [1]:
import pandas as pd
from utils import get_path, group_cols
from datatable import dt, f, by

# Descrição

Neste notebook iremos formar a tabela deslocamento, com origem e destino.

# Gerando tabela

In [2]:
def get_count_df(path_proc, ref):
  cols_proc = {
    'bienio': 'bienio', 'parto_normal': 'parto_normal',
    ref: 'codigo', 'criticidade': 'criticidade',
    'count': 'count', ...: None}
  df_proc = dt.fread(path_proc, columns=cols_proc)
  df_proc[:, dt.update(criticidade = f.criticidade != 0)]
  df_proc = df_proc[:, dt.sum(f.count), by(df_proc.names[:-1])]
  df_proc_total = df_proc[:, f[:].remove(f.criticidade)]
  df_proc_total = df_proc_total[:, dt.sum(f.count), by(df_proc_total.names[:-1])]
  df_proc_total.names = {'count': 'count_total'}
  df_proc_critic = df_proc[f.criticidade == 1, f[:].remove(f.criticidade)]
  df_proc_critic = df_proc_critic[:, dt.sum(f.count), by(df_proc_critic.names[:-1])]
  df_proc_critic.names = {'count': 'count_critic'}
  df_proc_critic.key = df_proc_critic.names[:-1]
  df_counts = df_proc_total[:, :, dt.join(df_proc_critic)]
  df_counts['count_critic'] = dt.Frame([
    0 if c == None else c for c in df_counts['count_critic'].to_list()[0]])
  df_counts['referencial'] = ref
  cols_order = ['bienio', 'parto_normal', 'codigo', 'referencial', 'count_total', 'count_critic']
  df_counts = df_counts[:, cols_order]
  df_counts = df_counts.sort(f.bienio, f.parto_normal, -f.count_critic)
  return df_counts

In [3]:
path_proc = get_path('GESTANTES', 'procs_bienio.csv.gzip')
refs = ['origem', 'destino']
df_counts = dt.rbind(*[get_count_df(path_proc, ref) for ref in refs])
df_counts

,bienio,parto_normal,codigo,referencial,count_total,count_critic
,▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,0,0,NT/PA/15006/150080,origem,6816,2638
1,0,0,ND/PE/26010/261160,origem,5177,2605
2,0,0,SD/ES/32002/320500,origem,4080,2263
3,0,0,SD/RJ/33005/330350,origem,2730,1951
4,0,0,CO/MT/51002/510840,origem,2061,1873
5,0,0,SD/ES/32002/320130,origem,3685,1525
6,0,0,ND/PE/26010/260790,origem,3274,1479
7,0,0,SD/RJ/33005/330455,origem,11372,1347
8,0,0,SD/ES/32002/320520,origem,3171,1215


# Exportando tabela

In [4]:
output_counts = get_path('GESTANTES', 'contagens.csv.gzip')
df_counts.to_csv(output_counts)

In [5]:
dt.fread(output_counts)

,bienio,parto_normal,codigo,referencial,count_total,count_critic
,▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,0,0,NT/PA/15006/150080,origem,6816,2638
1,0,0,ND/PE/26010/261160,origem,5177,2605
2,0,0,SD/ES/32002/320500,origem,4080,2263
3,0,0,SD/RJ/33005/330350,origem,2730,1951
4,0,0,CO/MT/51002/510840,origem,2061,1873
5,0,0,SD/ES/32002/320130,origem,3685,1525
6,0,0,ND/PE/26010/260790,origem,3274,1479
7,0,0,SD/RJ/33005/330455,origem,11372,1347
8,0,0,SD/ES/32002/320520,origem,3171,1215
